In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
import random


from numpy import random
from sklearn.preprocessing import OrdinalEncoder
from IPython import display


#tf.compat.v1.disable_eager_execution()

#%matplotlib inline
random.seed(246)

In [ ]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("valid.csv")

In [ ]:
max_journ_len = 16 #as specified in prep_data.jpynb

# Next step: transform to tensor

In [ ]:
def mta2tensor(df, max_journ_len):
    colx = df.shape[1] - 1
    df_transaction = df['transaction']
    data = df.drop('transaction', axis=1)
    grous = df.groupby('journey_id')
    x = []
    y = []
    cj_count = 1
    for i in df['journey_id'].unique():
        x1 = grous.get_group(i)

        x1 = x1.drop(['journey_id'], axis=1)
        x1 = x1.values.tolist()

        y1 = []
        y_prop = df_transaction.loc[grous.get_group(i).index]
        y_prop = y_prop.values.tolist()
        if y_prop[0]==1:
            y1 = 1
        else: y1 = 0

        #for l in range(len(y_prop)):
           #y1.append([y_prop[l]])

        for j in range(max_journ_len - len(x1)):

            x1.append([0] * colx)  # 52 is number of columns without journey_id an transaction
            #y1.append([0])

        x.append(x1)
        y.append(y1)
        cj_count = cj_count + 1

    return x, y
    #tf.convert_to_tensor(x), tf.convert_to_tensor(y)


In [ ]:
X_test, Y_test = mta2tensor(df_train, max_journ_len)
X_train, Y_train = mta2tensor(df_test, max_journ_len)
#X_valid, Y_valid = mta2tensor(df_valid, max_journ_len)

In [ ]:
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
train_dataset = train_dataset.batch(batch_size)

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
test_dataset = test_dataset.batch(batch_size)

In [ ]:
time_steps = max_journ_len
input_dim = df_train.shape[1] - 1
num_classes = 2

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.GRU(64, return_sequences=True, input_shape=(time_steps, input_dim)),
    tf.keras.layers.GRU(32),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [ ]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
class LossHistory(tf.keras.callbacks.Callback):
    def __init__(self):
        self.train_losses = []
        self.test_losses = []

    def on_epoch_end(self, epoch, logs=None):
        self.train_losses.append(logs.get('loss'))
        self.test_losses.append(logs.get('val_loss'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Create instances of the custom callback
loss_history = LossHistory()

# Train the model with the custom callback
num_epochs = 100
model.fit(test_dataset, epochs=num_epochs, validation_data=train_dataset, callbacks=[loss_history])

In [ ]:
plt.plot(loss_history.train_losses, '-b', label='Train loss')
plt.plot(loss_history.test_losses, '-r', label='Test loss')
plt.legend(loc=0)
plt.title('Loss')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.show()